In [989]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import branca
import folium
from folium import IFrame
from folium import FeatureGroup, LayerControl, Map, Marker, plugins
import base64

from math import pi

# code reference at blog.prototypr.io/interactive-maps-with-python-part-1-aa1563dbe5a9
# transit districts source: www1.nyc.gov/assets/nypd/downloads/pdf/maps/transit-district-map.pdf

In [990]:
# load csv of all station information
subway_data = pd.read_csv("Stations.csv")

# need to get lat/long of all stations to draw circle
# need to figure out how to make interactable (default pics)
subway_data

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan
5,6,6,R08,BMT,Astoria,39 Av,Q,N W,Elevated,40.752882,-73.932755,Astoria - Ditmars Blvd,Manhattan
6,7,613,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.762660,-73.967258,Queens,Downtown & Brooklyn
7,8,8,R13,BMT,Astoria,5 Av/59 St,M,N W R,Subway,40.764811,-73.973347,Queens,Downtown & Brooklyn
8,9,9,R14,BMT,Broadway - Brighton,57 St - 7 Av,M,N Q R W,Subway,40.764664,-73.980658,Uptown & Queens,Downtown & Brooklyn
9,10,10,R15,BMT,Broadway - Brighton,49 St,M,N R W,Subway,40.759901,-73.984139,Uptown & Queens,Downtown & Brooklyn


In [991]:
locations = subway_data.groupby("GTFS Latitude").first()
locations = locations.loc[:, ["GTFS Longitude",
                              "Stop Name",
                              "Borough"]]
locations.groupby('Stop Name')

In [992]:
highCountStations = ['59 St - Columbus Circle','Franklin St','125 St','14 St - Union Sq','Tremont Av','3 Av - 149 St','Flushing - Main St','Broad Channel','Jay St - MetroTech','Union St','Atlantic Av','Newkirk Av']
lowCountStations = ['79 St','Wall St','103 St','Spring St','Fordham Rd','Freeman St','Elmhurst Av','Beach 98 St','Franklin Av','Kingston Av','Montrose Av','Brighton Beach']

In [993]:
highLocations = locations[locations['Stop Name'].isin(highCountStations)]
highLocations.drop_duplicates(subset ='Stop Name',keep='first',inplace=True)

/Users/dylanfosgett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [994]:
highSpots = ['High','High','High','High','High','High','High','High','High','High','High','High']
highLocations['Cop Sightings'] = highSpots
highLocations['Transit District'] = [23,34,33,32,30,2,4,20,1,3,12,11]

/Users/dylanfosgett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/dylanfosgett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [995]:
lowCountStations.sort()

In [996]:
lowLocations = locations[locations['Stop Name'].isin(lowCountStations)]
lowLocations.sort_values(by='Stop Name')
lowLocations.drop_duplicates(subset='Stop Name',keep='last',inplace=True)
lowSpots = ['Low','Low','Low','Low','Low','Low','Low','Low','Low','Low','Low','Low']
lowLocations['Cop Sightings'] = lowSpots
lowLocations['Transit District'] = [34,23,32,30,2,33,4,20,1,3,12,11]

/Users/dylanfosgett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/dylanfosgett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/dylanfosgett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [997]:
lowHighLocations = [lowLocations,highLocations]
lowHigh = pd.concat(lowHighLocations)
lowHigh = lowHigh.sort_values(by='Stop Name',ascending=True)
lowHigh

,GTFS Longitude,Stop Name,Borough,Cop Sightings,Transit District
GTFS Latitude,,,,,
40.799446,-73.968379,103 St,M,Low,3
40.804138,-73.937594,125 St,M,High,3
40.734673,-73.989951,14 St - Union Sq,M,High,4
40.816109,-73.917757,3 Av - 149 St,Bx,High,12
40.768247,-73.981929,59 St - Columbus Circle,M,High,1
40.783934,-73.979917,79 St,M,Low,1
40.675345,-73.903097,Atlantic Av,Bk,High,33
40.585307,-73.820558,Beach 98 St,Q,Low,23
40.577621,-73.961376,Brighton Beach,Bk,Low,34


# Plotting Functions

In [998]:
# Universal Variables for all plot functions
radius = 15

In [999]:
# plot circles at each station
def plot_stations_colored_by_community_district(station):
    #folium.TileLayer('stamenterrain').add_to(folium_map)
    #folium.LayerControl().add_to(folium_map)
    
    feature_group = FeatureGroup(name='Community District',show=False)
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        #print(row.isin(lowLocations['Stop Name']))
        # number of spotted officers plotted
        
        # generate popup message shown on click
        popup_text="Stop Name: {}"
        popup_text=(row["Stop Name"])
        
        # tooltip is text displayed when only hovering over popup
        tooltip_text = "Stop Name: {}<br> Borough: {}<br> Police Sightings: {}<br>"
        tooltip_text = tooltip_text.format(row["Stop Name"], row["Borough"],row["Cop Sightings"])
        # radius of circles
        #radius = 10
        # color of circles (can change conditionally)
        
        # color for low Locations is: purple
        if(row["Borough"] == 'Q'):
            color = '#ff0000'
        if(row["Borough"] == 'Bk'):
            color = '#00ff00'
        if(row["Borough"] == 'Bx'):
            color = '#0000ff'
        if(row["Borough"] == 'M'):
            color = '#ffff00'
        if(row["Borough"] == 'SI'):
            color = '#000000'
        folium.CircleMarker(location=(index, row["GTFS Longitude"]),
                            popup=popup_text,
                            radius=radius,
                            color=color,
                            fill=True,
                            fill_color=color,
                            fill_opacity=1,
                            tooltip=tooltip_text).add_to(feature_group)
    return feature_group

In [1000]:
# plot circles at each station
# colored by specific stations
def plot_stations_colored_by_transit_district(station):
    #folium.TileLayer('stamenterrain').add_to(folium_map)
    #folium.LayerControl().add_to(folium_map)
    
    feature_group = FeatureGroup(name='Transit District',show=True)
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        # tooltip is text displayed when only hovering over popup
        tooltip_text = "Stop Name: {}<br> Transit District: {}<br> Police Sightings: {}<br>"
        tooltip_text = tooltip_text.format(row["Stop Name"], row["Transit District"],row["Cop Sightings"])
        
        # color of circles (can change conditionally)
        if(row['Transit District']==3):
            #brown
            color = '#964b00'
        if(row['Transit District']==2):
            #green
            color = '#00ff00'
        if(row['Transit District']==11):
            #pink
            color = '#ff69b4'
        if(row['Transit District']==4):
            #black
            color = '#000000'
        if(row['Transit District']==1):
            #red
            color = '#ff0000'
        if(row['Transit District']==20):
            #blue
            color = '#0000ff'
        if(row['Transit District']==32):
            #violet
            color = '#8a2be2'
        if(row['Transit District']==23):
            #orange
            color = '#ffa500'
        if(row['Transit District']==30):
            #teal
            color = '#008080'
        if(row['Transit District']==34):
            #brown
            color = '#b5651d'
        if(row['Transit District']==12):
            #yellow
            color = '#ffff00'
        if(row['Transit District']==33):
            #purple
            color = '#800080'

        # generate popup message shown on click
        popup_text = 'Click a group in the top right for data by race/gender/age'
        
        #genereate circle popups
        folium.CircleMarker(location=(index, row["GTFS Longitude"]),
                            popup=popup_text,
                            radius=radius,
                            color=color,
                            fill=True,
                            fill_color=color,
                            fill_opacity=1,
                            tooltip=tooltip_text).add_to(feature_group)
    
    return feature_group

### Plot Functions - Arrests (age/race/gender)

In [1001]:
# plot circles at each station
# colored by specific stations
def plot_stations_age(station):
    #folium.TileLayer('stamenterrain').add_to(folium_map)
    #folium.LayerControl().add_to(folium_map)
    
    feature_group = FeatureGroup(name='Arrests by Age',show=False)
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        
        
        # POPUP GENERATOR (COPY & PASTE)
        if(row['Stop Name']=='103 St'):
            file = './popups/A103St.png' # Fill in A with A/R/G
        if(row['Stop Name']=='125 St'):
            file = './popups/A125St.png'
        if(row['Stop Name']=='14 St - Union Sq'):
            file = './popups/A14Stunionsq.png'
        if(row['Stop Name']=='3 Av - 149 St'):
            file = './popups/A3Av149St.png'
        if(row['Stop Name']=='59 St - Columbus Circle'):
            file = './popups/A59stcolumbuscircle.png'
        if(row['Stop Name']=='79 St'):
            file = './popups/A79St.png'
        if(row['Stop Name']=='Atlantic Av'):
            file = './popups/Aatlanticav.png'
        if(row['Stop Name']=='Beach 98 St'):
            file = './popups/Abeach98St.png'
        if(row['Stop Name']=='Brighton Beach'):
            file = './popups/Abrightonbeach.png'
        if(row['Stop Name']=='Broad Channel'):
            file = './popups/Abroadchannel.png'
        if(row['Stop Name']=='Elmhurst Av'):
            file = './popups/Aelmhurstav.png'
        if(row['Stop Name']=='Flushing - Main St'):
            file = './popups/Aflushingmainst.png'
            
            
        if(row['Stop Name']=='Fordham Rd'):
            file = './popups/Afordhamrd.png' # Fill in A with A/R/G
        if(row['Stop Name']=='Franklin Av'):
            file = './popups/Afranklinav.png'
        if(row['Stop Name']=='Franklin St'):
            file = './popups/Afranklinst.png'
        if(row['Stop Name']=='Freeman St'):
            file = './popups/Afreemanst.png'
        if(row['Stop Name']=='Jay St - MetroTech'):
            file = './popups/Ajayst.png'
        if(row['Stop Name']=='Kingston Av'):
            file = './popups/Akingstonav.png'
        if(row['Stop Name']=='montrose Av'):
            file = './popups/Amontroseav.png'
        if(row['Stop Name']=='Newkirk Av'):
            file = './popups/Anewkirkav.png'
        if(row['Stop Name']=='Spring St'):
            file = './popups/Aspringst.png'
        if(row['Stop Name']=='Tremont Av'):
            file = './popups/Atremontav.png'
        if(row['Stop Name']=='Union St'):
            file = './popups/Aunionst.png'
        if(row['Stop Name']=='Wall St'):
            file = './popups/Awallst.png'
        
        
        # tooltip is teAt displayed when only hovering over popup
        tooltip_teAt = "Stop Name: {}<br> Transit District: {}<br> Police Sightings: {}<br>"
        tooltip_teAt = tooltip_teAt.format(row["Stop Name"], row["Transit District"],row["Cop Sightings"])
        # radius of circles
        #radius = 10
        
        # color of circles (can change conditionally)
        if(row['Transit District']==3):
            #brown
            color = '#964b00'
        if(row['Transit District']==2):
            #green
            color = '#00ff00'
        if(row['Transit District']==11):
            #pink
            color = '#ff69b4'
        if(row['Transit District']==4):
            #black
            color = '#000000'
        if(row['Transit District']==1):
            #red
            color = '#ff0000'
        if(row['Transit District']==20):
            #blue
            color = '#0000ff'
        if(row['Transit District']==32):
            #violet
            color = '#8a2be2'
        if(row['Transit District']==23):
            #orange
            color = '#ffa500'
        if(row['Transit District']==30):
            #teal
            color = '#008080'
        if(row['Transit District']==34):
            #brown
            color = '#b5651d'
        if(row['Transit District']==12):
            #yellow
            color = '#ffff00'
        if(row['Transit District']==33):
            #purple
            color = '#800080'
        # generate popup message shown on click
        encoded = base64.b64encode(open(file, 'rb').read())
        html='<img src="data:image/png;base64,{}">'.format
        iframe = IFrame(html(encoded.decode('UTF-8')), width=(800), height=(300))
        type(iframe)
        test_popup = folium.Popup(iframe,maA_width='100%')
        
        folium.CircleMarker(location=(index, row["GTFS Longitude"]),
                            popup=test_popup,
                            radius=radius,
                            color='#ffffff',
                            fill=True,
                            fill_color=color,
                            fill_opacity=1,
                            tooltip=tooltip_teAt).add_to(feature_group)
    
    return feature_group

In [1002]:
# plot circles at each station
# colored by specific stations
def plot_stations_race(station):
    #folium.TileLayer('stamenterrain').add_to(folium_map)
    #folium.LayerControl().add_to(folium_map)
    
    feature_group_race_arrests = FeatureGroup(name='Arrests by Race',show=False)
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        # POPUP GENERATOR (COPY & PASTE)
        if(row['Stop Name']=='103 St'):
            file = './popups/R103St.png' # Fill in R with A/R/G
        if(row['Stop Name']=='125 St'):
            file = './popups/R125St.png'
        if(row['Stop Name']=='14 St - Union Sq'):
            file = './popups/R14Stunionsq.png'
        if(row['Stop Name']=='3 Av - 149 St'):
            file = './popups/R3Av149St.png'
        if(row['Stop Name']=='59 St - Columbus Circle'):
            file = './popups/R59stcolumbuscircle.png'
        if(row['Stop Name']=='79 St'):
            file = './popups/R79St.png'
        if(row['Stop Name']=='Atlantic Av'):
            file = './popups/Ratlanticav.png'
        if(row['Stop Name']=='Beach 98 St'):
            file = './popups/Rbeach98St.png'
        if(row['Stop Name']=='Brighton Beach'):
            file = './popups/Rbrightonbeach.png'
        if(row['Stop Name']=='Broad Channel'):
            file = './popups/Rbroadchannel.png'
        if(row['Stop Name']=='Elmhurst Av'):
            file = './popups/Relmhurstav.png'
        if(row['Stop Name']=='Flushing - Main St'):
            file = './popups/Rflushingmainst.png'
            
        if(row['Stop Name']=='Fordham Rd'):
            file = './popups/Rfordhamrd.png' # Fill in A with A/R/G
        if(row['Stop Name']=='Franklin Av'):
            file = './popups/Rfranklinav.png'
        if(row['Stop Name']=='Franklin St'):
            file = './popups/Rfranklinst.png'
        if(row['Stop Name']=='Freeman St'):
            file = './popups/Rfreemanst.png'
        if(row['Stop Name']=='Jay St - MetroTech'):
            file = './popups/Rjayst.png'
        if(row['Stop Name']=='Kingston Av'):
            file = './popups/Rkingstonav.png'
        if(row['Stop Name']=='montrose Av'):
            file = './popups/Rmontroseav.png'
        if(row['Stop Name']=='Newkirk Av'):
            file = './popups/Rnewkirkav.png'
        if(row['Stop Name']=='Spring St'):
            file = './popups/Rspringst.png'
        if(row['Stop Name']=='Tremont Av'):
            file = './popups/Rtremontav.png'
        if(row['Stop Name']=='Union St'):
            file = './popups/Runionst.png'
        if(row['Stop Name']=='Wall St'):
            file = './popups/Rwallst.png'
        
        # tooltip is text displayed when only hovering over popup
        tooltip_race_arrests = "Stop Name: {}<br> Transit District: {}<br> Police Sightings: {}<br>"
        tooltip_race_arrests = tooltip_race_arrests.format(row["Stop Name"], row["Transit District"],row["Cop Sightings"])
        # radius of circles
        #radius = 10
        
        # color of circles (can change conditionally)
        if(row['Transit District']==3):
            #brown
            color = '#964b00'
        if(row['Transit District']==2):
            #green
            color = '#00ff00'
        if(row['Transit District']==11):
            #pink
            color = '#ff69b4'
        if(row['Transit District']==4):
            #black
            color = '#000000'
        if(row['Transit District']==1):
            #red
            color = '#ff0000'
        if(row['Transit District']==20):
            #blue
            color = '#0000ff'
        if(row['Transit District']==32):
            #violet
            color = '#8a2be2'
        if(row['Transit District']==23):
            #orange
            color = '#ffa500'
        if(row['Transit District']==30):
            #teal
            color = '#008080'
        if(row['Transit District']==34):
            #brown
            color = '#b5651d'
        if(row['Transit District']==12):
            #yellow
            color = '#ffff00'
        if(row['Transit District']==33):
            #purple
            color = '#800080'
        # generate popup message shown on click
        race_arrests_encoded = base64.b64encode(open(file, 'rb').read())
        html='<img src="data:image/png;base64,{}">'.format
        iframe = IFrame(html(race_arrests_encoded.decode('UTF-8')), width=(799), height=(300))
        type(iframe)
        arrests_popup = folium.Popup(iframe,max_width='20%')
        
        folium.CircleMarker(location=(index, row["GTFS Longitude"]),
                            popup=arrests_popup,
                            radius=radius,
                            color='#999999',
                            fill=True,
                            fill_color=color,
                            fill_opacity=1,
                            tooltip=tooltip_race_arrests).add_to(feature_group_race_arrests)
    
    return feature_group_race_arrests

In [1003]:
# plot circles at each station
# colored by specific stations
def plot_stations_gender(station):
    #folium.TileLayer('stamenterrain').add_to(folium_map)
    #folium.LayerControl().add_to(folium_map)
    
    feature_group_gender_arrests = FeatureGroup(name='Arrests by Gender',show=False)
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        # POPUP GENERATOR (COPY & PASTE)
        if(row['Stop Name']=='103 St'):
            file = './popups/G103St.png' # Fill in G with A/R/G
        if(row['Stop Name']=='125 St'):
            file = './popups/G125St.png'
        if(row['Stop Name']=='14 St - Union Sq'):
            file = './popups/G14Stunionsq.png'
        if(row['Stop Name']=='3 Av - 149 St'):
            file = './popups/G3Av149St.png'
        if(row['Stop Name']=='59 St - Columbus Circle'):
            file = './popups/G59stcolumbuscircle.png'
        if(row['Stop Name']=='79 St'):
            file = './popups/G79St.png'
        if(row['Stop Name']=='Atlantic Av'):
            file = './popups/Gatlanticav.png'
        if(row['Stop Name']=='Beach 98 St'):
            file = './popups/Gbeach98St.png'
        if(row['Stop Name']=='Brighton Beach'):
            file = './popups/Gbrightonbeach.png'
        if(row['Stop Name']=='Broad Channel'):
            file = './popups/Gbroadchannel.png'
        if(row['Stop Name']=='Elmhurst Av'):
            file = './popups/Gelmhurstav.png'
        if(row['Stop Name']=='Flushing - Main St'):
            file = './popups/Gflushingmainst.png'
            
        if(row['Stop Name']=='Fordham Rd'):
            file = './popups/Gfordhamrd.png' # Fill in A with A/R/G
        if(row['Stop Name']=='Franklin Av'):
            file = './popups/Gfranklinav.png'
        if(row['Stop Name']=='Franklin St'):
            file = './popups/Gfranklinst.png'
        if(row['Stop Name']=='Freeman St'):
            file = './popups/Gfreemanst.png'
        if(row['Stop Name']=='Jay St - MetroTech'):
            file = './popups/Gjayst.png'
        if(row['Stop Name']=='Kingston Av'):
            file = './popups/Gkingstonav.png'
        if(row['Stop Name']=='montrose Av'):
            file = './popups/Gmontroseav.png'
        if(row['Stop Name']=='Newkirk Av'):
            file = './popups/Gnewkirkav.png'
        if(row['Stop Name']=='Spring St'):
            file = './popups/Gspringst.png'
        if(row['Stop Name']=='Tremont Av'):
            file = './popups/Gtremontav.png'
        if(row['Stop Name']=='Union St'):
            file = './popups/Gunionst.png'
        if(row['Stop Name']=='Wall St'):
            file = './popups/Gwallst.png'
        
        # tooltip is text displayed when only hovering over popup
        tooltip_gender_arrests = "Stop Name: {}<br> Transit District: {}<br> Police Sightings: {}<br>"
        tooltip_gender_arrests = tooltip_gender_arrests.format(row["Stop Name"], row["Transit District"],row["Cop Sightings"])
        # radius of circles
        #radius = 10
        
        # color of circles (can change conditionally)
        if(row['Transit District']==3):
            #brown
            color = '#964b00'
        if(row['Transit District']==2):
            #green
            color = '#00ff00'
        if(row['Transit District']==11):
            #pink
            color = '#ff69b4'
        if(row['Transit District']==4):
            #black
            color = '#000000'
        if(row['Transit District']==1):
            #red
            color = '#ff0000'
        if(row['Transit District']==20):
            #blue
            color = '#0000ff'
        if(row['Transit District']==32):
            #violet
            color = '#8a2be2'
        if(row['Transit District']==23):
            #orange
            color = '#ffa500'
        if(row['Transit District']==30):
            #teal
            color = '#008080'
        if(row['Transit District']==34):
            #brown
            color = '#b5651d'
        if(row['Transit District']==12):
            #yellow
            color = '#ffff00'
        if(row['Transit District']==33):
            #purple
            color = '#800080'
        # generate popup message shown on click
        race_arrests_encoded = base64.b64encode(open(file, 'rb').read())
        html='<img src="data:image/png;base64,{}">'.format
        iframe = IFrame(html(race_arrests_encoded.decode('UTF-8')), width=(800), height=(350))
        gender_arrests_popup = folium.Popup(iframe,max_width='100%')
        
        folium.CircleMarker(location=(index, row["GTFS Longitude"]),
                            popup=gender_arrests_popup,
                            radius=radius,
                            color='#444444',
                            fill=True,
                            fill_color=color,
                            fill_opacity=1,
                            tooltip=tooltip_gender_arrests).add_to(feature_group_gender_arrests)
    
    return feature_group_gender_arrests

### Plot Functions - Summons (age/race/gender)

In [1004]:
# plot circles at each station
# colored by specific stations
def plot_stations_colored_by_age_summons(station):
    #folium.TileLayer('stamenterrain').add_to(folium_map)
    #folium.LayerControl().add_to(folium_map)
    
    feature_group_age_summons = FeatureGroup(name='Summons by age',show=False)
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        # POPUP GENERATOR (COPY & PASTE)
        if(row['Stop Name']=='103 St'):
            file = './popups/A103St.png' # Fill in A with A/R/G
        if(row['Stop Name']=='125 St'):
            file = './popups/A125St.png'
        if(row['Stop Name']=='14 St - Union Sq'):
            file = './popups/A14Stunionsq.png'
        if(row['Stop Name']=='3 Av - 149 St'):
            file = './popups/A3Av149St.png'
        if(row['Stop Name']=='59 St - Columbus Circle'):
            file = './popups/A59stcolumbuscircle.png'
        if(row['Stop Name']=='79 St'):
            file = './popups/A79St.png'
        if(row['Stop Name']=='Atlantic Av'):
            file = './popups/Aatlanticav.png'
        if(row['Stop Name']=='Beach 98 St'):
            file = './popups/Abeach98St.png'
        if(row['Stop Name']=='Brighton Beach'):
            file = './popups/Abrightonbeach.png'
        if(row['Stop Name']=='Broad Channel'):
            file = './popups/Abroadchannel.png'
        if(row['Stop Name']=='Elmhurst Av'):
            file = './popups/Aelmhurstav.png'
        if(row['Stop Name']=='Flushing - Main St'):
            file = './popups/Aflushingmainst.png'
        
        # tooltip is text displayed when only hovering over popup
        tooltip_age_summons = "Stop Name: {}<br> Transit District: {}<br> Police Sightings: {}<br>"
        tooltip_age_summons = tooltip_age_summons.format(row["Stop Name"], row["Transit District"],row["Cop Sightings"])
        # radius of circles
        #radius = 10
        
        # color of circles (can change conditionally)
        if(row['Transit District']==3):
            #brown
            color = '#964b00'
        if(row['Transit District']==2):
            #green
            color = '#00ff00'
        if(row['Transit District']==11):
            #pink
            color = '#ff69b4'
        if(row['Transit District']==4):
            #black
            color = '#000000'
        if(row['Transit District']==1):
            #red
            color = '#ff0000'
        if(row['Transit District']==20):
            #blue
            color = '#0000ff'
        if(row['Transit District']==32):
            #violet
            color = '#8a2be2'
        if(row['Transit District']==23):
            #orange
            color = '#ffa500'
        if(row['Transit District']==30):
            #teal
            color = '#008080'
        if(row['Transit District']==34):
            #brown
            color = '#b5651d'
        if(row['Transit District']==12):
            #yellow
            color = '#ffff00'
        if(row['Transit District']==33):
            #purple
            color = '#800080'
        # generate popup message shown on click
        encoded = base64.b64encode(open(file, 'rb').read())
        html='<img src="data:image/png;base64,{}">'.format
        iframe = IFrame(html(encoded.decode('UTF-8')), width=(800), height=(350))
        age_summons_popup = folium.Popup(iframe,max_width='100%')
        
        folium.CircleMarker(location=(index, row["GTFS Longitude"]),
                            popup=age_summons_popup,
                            radius=radius,
                            color='#ffffff',
                            fill=True,
                            fill_color=color,
                            fill_opacity=1,
                            tooltip=tooltip_age_summons).add_to(feature_group_age_summons)
    
    return feature_group_age_summons

In [1005]:
# plot circles at each station
# colored by specific stations
def plot_stations_colored_by_race_summons(station):
    #folium.TileLayer('stamenterrain').add_to(folium_map)
    #folium.LayerControl().add_to(folium_map)
    
    feature_group_race_summons = FeatureGroup(name='Summons by Race',show=False)
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        # POPUP GENERATOR (COPY & PASTE)
        if(row['Stop Name']=='103 St'):
            file = './popups/R103St.png' # Fill in R with A/R/G
        if(row['Stop Name']=='125 St'):
            file = './popups/R125St.png'
        if(row['Stop Name']=='14 St - Union Sq'):
            file = './popups/R14Stunionsq.png'
        if(row['Stop Name']=='3 Av - 149 St'):
            file = './popups/R3Av149St.png'
        if(row['Stop Name']=='59 St - Columbus Circle'):
            file = './popups/R59stcolumbuscircle.png'
        if(row['Stop Name']=='79 St'):
            file = './popups/R79St.png'
        if(row['Stop Name']=='Atlantic Av'):
            file = './popups/Ratlanticav.png'
        if(row['Stop Name']=='Beach 98 St'):
            file = './popups/Rbeach98St.png'
        if(row['Stop Name']=='Brighton Beach'):
            file = './popups/Rbrightonbeach.png'
        if(row['Stop Name']=='Broad Channel'):
            file = './popups/Rbroadchannel.png'
        if(row['Stop Name']=='Elmhurst Av'):
            file = './popups/Relmhurstav.png'
        if(row['Stop Name']=='Flushing - Main St'):
            file = './popups/Rflushingmainst.png'
        
        # tooltip is text displayed when only hovering over popup
        tooltip_race_summons = "Stop Name: {}<br> Transit District: {}<br> Police Sightings: {}<br>"
        tooltip_race_summons = tooltip_race_summons.format(row["Stop Name"], row["Transit District"],row["Cop Sightings"])
        # radius of circles
        #radius = 10
        
        # color of circles (can change conditionally)
        if(row['Transit District']==3):
            #brown
            color = '#964b00'
        if(row['Transit District']==2):
            #green
            color = '#00ff00'
        if(row['Transit District']==11):
            #pink
            color = '#ff69b4'
        if(row['Transit District']==4):
            #black
            color = '#000000'
        if(row['Transit District']==1):
            #red
            color = '#ff0000'
        if(row['Transit District']==20):
            #blue
            color = '#0000ff'
        if(row['Transit District']==32):
            #violet
            color = '#8a2be2'
        if(row['Transit District']==23):
            #orange
            color = '#ffa500'
        if(row['Transit District']==30):
            #teal
            color = '#008080'
        if(row['Transit District']==34):
            #brown
            color = '#b5651d'
        if(row['Transit District']==12):
            #yellow
            color = '#ffff00'
        if(row['Transit District']==33):
            #purple
            color = '#800080'
        # generate popup message shown on click
        encoded = base64.b64encode(open(file, 'rb').read())
        html='<img src="data:image/png;base64,{}">'.format
        iframe = IFrame(html(encoded.decode('UTF-8')), width=(800), height=(350))
        race_summons_popup = folium.Popup(iframe,max_width='100%')
        
        folium.CircleMarker(location=(index, row["GTFS Longitude"]),
                            popup=race_summons_popup,
                            radius=radius,
                            color='#999999',
                            fill=True,
                            fill_color=color,
                            fill_opacity=1,
                            tooltip=tooltip_race_summons).add_to(feature_group_race_summons)
    
    return feature_group_race_summons

In [1006]:
# plot circles at each station
# colored by specific stations
def plot_stations_colored_by_gender_summons(station):
    #folium.TileLayer('stamenterrain').add_to(folium_map)
    #folium.LayerControl().add_to(folium_map)
    
    feature_group_gender_summons = FeatureGroup(name='Summons by Gender',show=False)
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        # POPUP GENERATOR (COPY & PASTE)
        if(row['Stop Name']=='103 St'):
            file = './popups/G103St.png' # Fill in G with A/R/G
        if(row['Stop Name']=='125 St'):
            file = './popups/G125St.png'
        if(row['Stop Name']=='14 St - Union Sq'):
            file = './popups/G14Stunionsq.png'
        if(row['Stop Name']=='3 Av - 149 St'):
            file = './popups/G3Av149St.png'
        if(row['Stop Name']=='59 St - Columbus Circle'):
            file = './popups/G59stcolumbuscircle.png'
        if(row['Stop Name']=='79 St'):
            file = './popups/G79St.png'
        if(row['Stop Name']=='Atlantic Av'):
            file = './popups/Gatlanticav.png'
        if(row['Stop Name']=='Beach 98 St'):
            file = './popups/Gbeach98St.png'
        if(row['Stop Name']=='Brighton Beach'):
            file = './popups/Gbrightonbeach.png'
        if(row['Stop Name']=='Broad Channel'):
            file = './popups/Gbroadchannel.png'
        if(row['Stop Name']=='Elmhurst Av'):
            file = './popups/Gelmhurstav.png'
        if(row['Stop Name']=='Flushing - Main St'):
            file = './popups/Gflushingmainst.png'
        
        # tooltip is text displayed when only hovering over popup
        tooltip_gender_summons = "Stop Name: {}<br> Transit District: {}<br> Police Sightings: {}<br>"
        tooltip_gender_summons = tooltip_gender_summons.format(row["Stop Name"], row["Transit District"],row["Cop Sightings"])
        # radius of circles
        #radius = 10
        
        # color of circles (can change conditionally)
        if(row['Transit District']==3):
            #brown
            color = '#964b00'
        if(row['Transit District']==2):
            #green
            color = '#00ff00'
        if(row['Transit District']==11):
            #pink
            color = '#ff69b4'
        if(row['Transit District']==4):
            #black
            color = '#000000'
        if(row['Transit District']==1):
            #red
            color = '#ff0000'
        if(row['Transit District']==20):
            #blue
            color = '#0000ff'
        if(row['Transit District']==32):
            #violet
            color = '#8a2be2'
        if(row['Transit District']==23):
            #orange
            color = '#ffa500'
        if(row['Transit District']==30):
            #teal
            color = '#008080'
        if(row['Transit District']==34):
            #brown
            color = '#b5651d'
        if(row['Transit District']==12):
            #yellow
            color = '#ffff00'
        if(row['Transit District']==33):
            #purple
            color = '#800080'
        # generate popup message shown on click
        encoded = base64.b64encode(open(file, 'rb').read())
        html='<img src="data:image/png;base64,{}">'.format
        iframe = IFrame(html(encoded.decode('UTF-8')), width=(800), height=(350))
        gender_summons_popup = folium.Popup(iframe,max_width='100%')
        
        folium.CircleMarker(location=(index, row["GTFS Longitude"]),
                            popup=gender_summons_popup,
                            radius=radius,
                            color='#444444',
                            fill=True,
                            fill_color=color,
                            fill_opacity=1,
                            tooltip=tooltip_gender_summons).add_to(feature_group_gender_summons)
    
    return feature_group_gender_summons

## Map Maker

In [1007]:
# call function plot_stations of each subway stop
#plot_stations(locations)

In [1008]:
# CREATES MAIN MAP
folium_map = folium.Map(location=[40.738, -73.98],
                            tiles=None)
folium.TileLayer('stamentoner',control=False).add_to(folium_map)
#create groups
age = plot_stations_age(lowHigh)
race = plot_stations_race(lowHigh)
gender = plot_stations_gender(lowHigh)

communityDistrict = plot_stations_colored_by_community_district(lowHigh)
transitDistrict = plot_stations_colored_by_transit_district(lowHigh)

#add groups to map
folium_map.add_child(age)
folium_map.add_child(race)
folium_map.add_child(gender)

folium_map.add_child(communityDistrict)
folium_map.add_child(transitDistrict)

folium.LayerControl(collapsed=True).add_to(folium_map)

In [1009]:
folium_map.save("Map.html")